BASIC CODE

In [1]:
#Installing required libraries
!pip install git+https://github.com/openai/whisper.git -q
!pip install ffmpeg -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
#Import required modules
import whisper
from google.colab import files

In [3]:
#Upload audio file
print("Upload your audio file: ")
uploaded=files.upload()

audio_file = list(uploaded.keys())[0]
print(f"Uploaded file: {audio_file}")

Upload your audio file: 


Saving WhatsApp Audio 2025-01-22 at 13.06.20_6de94f85-[AudioTrimmer.com].mp3 to WhatsApp Audio 2025-01-22 at 13.06.20_6de94f85-[AudioTrimmer.com] (1).mp3
Uploaded file: WhatsApp Audio 2025-01-22 at 13.06.20_6de94f85-[AudioTrimmer.com] (1).mp3


In [4]:
#Load Whisper model
model=whisper.load_model("small")

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


TRANSCRIPTION

In [5]:
#Transcribe audio
print("Transcribing audio...")
result = model.transcribe(audio_file, language='en')

Transcribing audio...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [6]:
#Display transcription
print("Transcription: ")
print(result["text"])

Transcription: 
 Hello and welcome to the course, Food Toxicology. My name is Professor Greg Muller and I'm the instructor for the course. I'd like to first welcome all the students that have undertaken this particular course of study. We hope that this course provides you with a good sound structural basis for an understanding and enhancing your knowledge in food toxicology. When we talk about food, we're talking about one of the most intimate relationships that we have with our environment. If you go back in history, you see that in fact, man has always questioned or challenged this particular interaction with his food system. This food system has presented many challenges in terms of potential or actual toxicosis. And in fact, through our history, we have found what is good to eat and what perhaps is poisonous or bad to eat.


QUESTION-ANSWER SECTION

In [7]:
!pip install chromadb sentence-transformers transformers

In [8]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from transformers import BartTokenizer, BartForConditionalGeneration

#Initialize ChromaDB and SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Updated ChromaDB Initialization
chroma_client = chromadb.Client(
    Settings(persist_directory="./chroma_db")
)

collection = chroma_client.get_or_create_collection(name="transcriptions")

#Loading BART model for text generation
bart_model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(bart_model_name)
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name)

#Add transcription chunks
transcription = """
Your transcription text here. Example: This is a sample transcription. It contains multiple sentences.
"""
transcription = result['text']

# Split transcription
chunks = transcription.split('. ')
for i, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk],
        metadatas=[{"chunk_id": i}],
        ids=[f"chunk_{i}"]
    )

#Question-answering function
def ask_question(question, top_k=3):
    question_embedding = embedder.encode(question).tolist()

    # Retrieve relevant chunks
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=top_k
    )

    #Combine chunks into context
    context = " ".join(results['documents'][0])

    #Generate answers using BART
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = bart_model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer

# Example Usage
question = "What is the main topic of the transcription?"
answer = ask_question(question)
print(f"Answer: {answer}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Answer: Food Toxicology course aims to provide a sound structural basis for an understanding and enhancing your knowledge in food toxicology. We hope that this course provides you with a good sound Structural basis for a good understanding of food Toxicology. The course is taught by Professor Greg Muller, a Professor of Food Toxicology at the University of Oxford.


In [9]:
# Example Usage
question = "Who is the professor or instructor"
answer = ask_question(question)
print(f"Answer: {answer}")

Answer: Professor Greg Muller is the instructor for the course. He hopes the course provides a good sound structural basis for an understanding and enhancing your knowledge in food toxicology. For more information on the course, visit the course's website or the course page on the university's Facebook page.


SUMMARIZATION

In [10]:
original_text=result['text']

In [11]:
#BART for summarization
bart_model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(bart_model_name)
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name)

#Summarize text
def summarize_text(text, max_input_length=1024, max_output_length=200):
    inputs = tokenizer(text, return_tensors="pt", max_length=max_input_length, truncation=True)
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=max_output_length, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Summarize the extracted text
if len(original_text) > 3000:
    summarized_text = summarize_text(original_text[:3000])
else:
    summarized_text = summarize_text(original_text)

# Print summarized text
print("\nSummarized Text:")
print(summarized_text)


Summarized Text:
Professor Greg Muller is the instructor for the Food Toxicology course. The course aims to provide students with a good sound structural basis for an understanding and enhancing their knowledge in food toxicology. In our history, we have found what is good to eat and what is poisonous or bad to eat.


GRADIO INTERFACE

In [12]:
!pip install gradio

In [13]:
import whisper
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from transformers import BartTokenizer, BartForConditionalGeneration
import gradio as gr

# Load Whisper model
whisper_model = whisper.load_model("small")

# Initialize ChromaDB and SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')
chroma_client = chromadb.Client(Settings(persist_directory="./chroma_db"))
collection = chroma_client.get_or_create_collection(name="transcriptions")

# Load BART model for text summarization
bart_model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(bart_model_name)
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name)

# Transcribe audio
def transcribe_audio(audio_file):
    result = whisper_model.transcribe(audio_file, language="en")
    transcription = result['text']

    # Split transcription into chunks and add to ChromaDB
    chunks = transcription.split('. ')
    for i, chunk in enumerate(chunks):
        collection.add(
            documents=[chunk],
            metadatas=[{"chunk_id": i}],
            ids=[f"chunk_{i}"]
        )
    return transcription

# Question answering
def ask_question(question):
    question_embedding = embedder.encode(question).tolist()

    # Retrieve relevant chunks
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=3
    )

    # Combine chunks into context
    context = " ".join(results['documents'][0])

    # Generate answers using BART
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = bart_model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Summarize text
def summarize_text(transcription):
    if len(transcription) > 3000:
        transcription = transcription[:3000]
    inputs = tokenizer(transcription, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=200, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Gradio Interface
def transcribe_and_interact(audio_file, question):
    transcription = transcribe_audio(audio_file)
    if question:
        answer = ask_question(question)
        return transcription, answer
    else:
        return transcription, "No question provided."

with gr.Blocks() as demo:
    gr.Markdown("# Audio Transcription, Q&A, and Summarization")

    with gr.Tab("Transcription and Q&A"):
        audio_input = gr.Audio(type="filepath", label="Upload Audio File")
        question_input = gr.Textbox(label="Ask a Question (Optional)", placeholder="Type your question here")
        transcription_output = gr.Textbox(label="Transcription", placeholder="Transcription will appear here")
        answer_output = gr.Textbox(label="Answer", placeholder="Answer will appear here")
        transcribe_button = gr.Button("Transcribe and Answer")

    with gr.Tab("Summarization"):
        text_input = gr.Textbox(label="Text for Summarization", placeholder="Enter transcription or text to summarize")
        summary_output = gr.Textbox(label="Summary", placeholder="Summary will appear here")
        summarize_button = gr.Button("Summarize")

    # Interactions
    transcribe_button.click(transcribe_and_interact, inputs=[audio_input, question_input], outputs=[transcription_output, answer_output])
    summarize_button.click(summarize_text, inputs=[text_input], outputs=summary_output)

# Launch the Gradio app
demo.launch()


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://689bc035b01ef5a643.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
!git clone https://github.com/krishnasharma3011/Classroom-Discussion-Summarizer.git


Cloning into 'Classroom-Discussion-Summarizer'...
fatal: could not read Username for 'https://github.com': No such device or address


In [15]:
!git clone https://github.com/krishnasharma2005/Classroom-Discussion-Summarizer.git


Cloning into 'Classroom-Discussion-Summarizer'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git clone https://github.com/krishnasharma2005/Classroom-Discussion-Summarizer.git